# Tarea: Hacer para N capas con un for

---
Imports

In [333]:
import numpy as np
import pandas as pd

---
Código de la Capa

In [334]:
class Capa:
    def __init__(self, num_neuronas, num_entradas, fun_act):
        self.peso_input = np.random.random((num_entradas+1, num_neuronas))
        self.fun_act = fun_act

    # Entra una matriz de 1 x n y sale una matriz de 1 x n
    def get_output(self, input):
        input = np.insert(input, 0, 1)
        if (input.shape[0] == self.peso_input.shape[0]):
            self.suma = np.matmul(input, self.peso_input)
            return self.step_function(self.suma)
        else: return "Ha habido un error: No coinciden el número de neuronas"

    # Decide qué hacer según la función de activación
    def step_function(self, input):
        if self.fun_act == 1:
            return self.sf_threshold(input)
        elif self.fun_act == 2:
            return self.sf_sigmoid(input)
        elif self.fun_act == 3:
            return self.sf_hyperbolic_tangent(input)
        elif self.fun_act == 4:
            return self.sf_relu(input)
        else:
            return "Ha habido un error: Función de activación incorrecta."

    def sf_threshold(self, input):
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    def sf_sigmoid(self, input):
        return 1/(1+np.exp(-input))

    def sf_hyperbolic_tangent(self, input):
        return (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input))

    def sf_relu(self, input):
        return np.maximum([0], input)

    # Función para mostrar los datos de la capa
    def print(self):
        print("Inputs: \n", self.peso_input)
        print("Input shape: \n", self.peso_input.shape)
        print("F Act: \n", self.fun_act)

---

In [335]:
class Red_Neuronal():
    def __init__(self,num_neuronas_capa, f_act_capa, num_inputs_1):
        self.lista_capas = []
        c = Capa(num_neuronas_capa[0], num_inputs_1, f_act_capa[0])
        self.lista_capas.append(c)
        for i in range(1, len(num_neuronas_capa)):
            c = Capa(num_neuronas_capa[i], num_neuronas_capa[i-1], f_act_capa[i])
            self.lista_capas.append(c)

    # Función para hacer los cálculos
    def test(self, inputs):
        for c in self.lista_capas:
            inputs = c.get_output(inputs)
        return inputs

    # Función para entrenar
    def train(self, input, out_esp , ratio):

        inputs, sumas = [], []
        for c in self.lista_capas:
            input = c.get_output(input)
            sumas.append(c.suma)
            inputs.append(input)

        # Derivadas de las funciones de activación
        df_act = self.f_act(sumas, 1)
        ecm = self.f_ecm(np.array(input), out_esp)
        decm = self.d_ecm(ecm)

        # Primera capa
        delta = decm * df_act # derivada f_coste * derivada f_activación

        pesos = self.lista_capas[-1].peso_input[1:]
        bias = self.lista_capas[-1].peso_input[0]

        self.lista_capas[-1].peso_input[1:] = pesos - delta * pesos * ratio
        self.lista_capas[-1].peso_input[0] = bias - delta * bias * ratio

        # Capas N
        for n in range(2, len(self.lista_capas)+1):
            pesos_tmp = pesos
            df_act = self.f_act(sumas, n)
            delta = delta @ pesos_tmp.transpose() * df_act # derivada f_coste * derivada f_activación * pesos anterior * derivada f_coste anterior

            pesos = self.lista_capas[-n].peso_input[1:]
            bias = self.lista_capas[-n].peso_input[0]

            # cambiar los self.pesos y self.bias
            self.lista_capas[-n].peso_input[1:] = pesos - np.array([inputs[-n] @ delta]).transpose() * ratio
            self.lista_capas[-n].peso_input[0] = bias - delta * bias * ratio

    # Función y derivada del ecm
    def f_ecm(self, r_esp, r_obt):
        ecm = ((r_esp - r_obt).sum()**2) / len(r_esp)
        return ecm

    def d_ecm(self, input):
        return np.sqrt(input)

    # Derivada según la función de activación
    def f_act(self, sumas, i):
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 2:
            return self.d_sigmoid(sumas[len(sumas)-i])
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 3:
            return self.d_hyperbolic_tangent(sumas[len(sumas)-i])
        if (self.lista_capas[len(self.lista_capas)-i].fun_act) == 4:
            return self.d_relu(sumas[len(sumas)-i])

    def d_sigmoid(self, input):
        return 1/(1+np.exp(-input)) * (1 - 1/(1+np.exp(-input)))

    def d_hyperbolic_tangent(self, input):
        return (2 / (np.exp(input) + np.exp(-input)))**2

    def d_relu(self, input):
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    # Función para mostrar los datos de la red neuronal
    def print(self):
        c = 1
        for capa in self.lista_capas:
            print("Capa c: ")
            capa.print()
            print("---------")
            c += 1

---
Pruebas

In [336]:
red_prueba = Red_Neuronal([4,2,3], [4,2,3], 4)
red_prueba.print()
red_prueba.train([[4, -2, 3, -5]], [[0.7, 0.9, 0.5]], 0.75)
print("\n\n\n")
red_prueba.print()

Capa c: 
Inputs: 
 [[0.10625812 0.3702407  0.01152055 0.81510877]
 [0.43352357 0.18995327 0.3446195  0.95322581]
 [0.93417915 0.84814704 0.24873553 0.9241633 ]
 [0.76876255 0.59028216 0.12585335 0.64125938]
 [0.29732635 0.16360492 0.4185365  0.14987453]]
Input shape: 
 (5, 4)
F Act: 
 4
---------
Capa c: 
Inputs: 
 [[0.94074987 0.44528561]
 [0.7548024  0.50305939]
 [0.65561158 0.98741931]
 [0.54801651 0.08687968]
 [0.71378788 0.83744604]]
Input shape: 
 (5, 2)
F Act: 
 2
---------
Capa c: 
Inputs: 
 [[0.94826184 0.03026863 0.80032419]
 [0.37777037 0.08995112 0.93338024]
 [0.37544468 0.76404608 0.75445555]]
Input shape: 
 (3, 3)
F Act: 
 3
---------




Capa c: 
Inputs: 
 [[0.10618635 0.36982536 0.01152055 0.81430146]
 [0.42863895 0.18506865 0.33973488 0.94834119]
 [0.92929452 0.84326242 0.24385091 0.91927867]
 [0.76387793 0.58539754 0.12096872 0.63637476]
 [0.29244172 0.15872029 0.41365188 0.14498991]]
Input shape: 
 (5, 4)
F Act: 
 4
---------
Capa c: 
Inputs: 
 [[0.94051591 0.4448525

for iteraciones in:  
    for datos_train:  
        train(datos)  
        if iteraciones%50 == 0:  
            test(datos)  
            error += nuevo_error  
        error_final = error / nº_datos_test  
        print(error)

In [337]:
inputs = pd.read_csv("03_inputs.csv")
outputs = pd.read_csv("03_outputs.csv")
red = Red_Neuronal([], [], 2)
for i in range(1, 2000):
    for datos_train:
        red.train(datos)
        if i % 50 == 0:
        for datos_test:
            red.test(datos)
            error += red.f_ecm()
        print(error)